In [ ]:
# Libraries

In [ ]:
import numpy as np

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
import seaborn as sns

In [ ]:
import cv2 as cv

In [ ]:
from sklearn import linear_model

In [ ]:
import warnings

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
plt.style.use('seaborn')

In [ ]:
# Helping Functions

In [ ]:
def plot_image(image):
    plt.imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))
    plt.show(block=True)

In [ ]:
def display_image(image):
    cv.imshow('image', image)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [ ]:
def MinMaxScalar(x, a=-1, b=1):
    N = (x - x.min()) * (b - a)
    D = (x.max() - x.min())
    return (N / D) + a

In [ ]:
# Activate Functions

In [ ]:
def sigmoid(t):
    return 1 / (1 + np.exp(-t))

In [ ]:
def relu(t):
    return np.maximum(0, t)

In [ ]:
def swish(t):
    return t / (1 + np.exp(-t))

In [ ]:
def softplus(t):
    return np.log(1 + np.exp(t))

In [ ]:
# Loadings

In [ ]:
def preprocess_line(image):
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    blur = cv.bilateralFilter(gray, 3, 75, 75)
    thresh = cv.adaptiveThreshold(blur, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 11, 2)
    k = 5
    kernal = np.ones((k, k), np.uint8)
    dilate = cv.dilate(thresh, kernal, iterations=1)
    k = 21
    kernal = np.ones((k, k), np.uint8)
    erode = cv.erode(dilate, kernal, iterations=1)
    return erode

In [ ]:
line1 = cv.imread('line1.jpg')

In [ ]:
preprocessed_line1 = preprocess_line(line1)

In [ ]:
plt.figure(figsize=(18, 10))
sns.scatterplot(np.arange(len(sum_line_scaled)), sum_line_scaled, alpha=0.5, label='scaled')
sns.scatterplot(np.arange(len(sum_line_scaled)), sigmoid(sum_line_scaled), alpha=0.7, label='sigmoid')
sns.scatterplot(np.arange(len(sum_line_scaled)), softplus(sigmoid(sum_line_scaled)), alpha=0.7, label='softplus')

In [ ]:
def word_extraction(preprocessed_line1, line1):
    sum_line = preprocessed_line1.sum(axis=0)
    sum_line_scaled = MinMaxScalar(sum_line, -1, 1)
    scaled_activated_sum = softplus(sigmoid(sum_line_scaled))
    l = scaled_activated_sum
    maximum = max(l)
    cutout_dip = []
    cutout_up = []
    for i in range(len(l)):
        if (i + 1) != len(l):
            if l[i] == maximum:
                if (l[i + 1] < maximum):
                    cutout_dip.append(i)
            if l[i] < maximum:
                if (l[i + 1] == maximum):
                    cutout_up.append(i)
    cutout_images = []
    gray_line1 = cv.cvtColor(line1, cv.COLOR_BGR2GRAY)
    for up, dip in zip(cutout_up, cutout_dip):
        temp = gray_line1[:, dip : up]
        cutout_images.append(temp)
    return cutout_images

In [ ]:
cutout_images = word_extraction(preprocessed_line1, line1)

In [ ]:
for i in range(len(cutout_images)):
    print('image number:', i + 1)
    plt.imshow(cv.cvtColor(cutout_images[i], cv.COLOR_BGR2RGB))
    plt.show(block=True)

In [ ]:
def preprocess_word(image):
    blur = cv.bilateralFilter(image, 3, 75, 75)
    thresh = cv.adaptiveThreshold(blur, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 11, 2)
    k = 5
    kernal = np.ones((k, k), np.uint8)
    dilate = cv.dilate(thresh, kernal, iterations=1)
    k = 15
    kernal = np.ones((k, k), np.uint8)
    erode = cv.erode(dilate, kernal, iterations=1)
    return erode